<a href="https://colab.research.google.com/github/AryaJeet1364/LangChain_Projects/blob/main/LangChainwHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project, I built a modular GenAI pipeline using LangChain and Hugging Face's Mistral-7B-Instruct model via hosted inference. I explored LangChain’s core components including PromptTemplate, LLMChain, SimpleSequentialChain, SequentialChain, and custom tool-based agents. By chaining prompts, I enabled multi-step reasoning, tool usage, and structured output generation.

I chose Hugging Face because it provides open-source, easily accessible, and free inference APIs for state-of-the-art models, making it ideal for experimentation. This project helped me understand how to build flexible, production-ready LLM workflows using modern tools as of June 2025.

## Checking for device

In [1]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

CUDA available: True
Device: cuda
GPU: Tesla T4


## Installation and Setup

In [2]:
!pip install langchain langchain-community langchain-huggingface
!pip install transformers torch sentence-transformers pandas faiss-cpu

In [3]:
!pip install -q langchain-experimental

In [4]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.agents import load_tools, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import CSVLoader
from langchain_experimental.agents import create_pandas_dataframe_agent
import os
import pandas as pd

In [5]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [6]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

## Prompt Templates

In [7]:
!pip install huggingface-hub

In [8]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain import PromptTemplate, LLMChain
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=128,
    temperature=0.5,
)

prompt = PromptTemplate(input_variables=["topic"], template="Tell me about {topic} in 2 lines.")
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({"topic": "quantum computing"}))

/tmp/ipython-input-8-3697858379.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-8-3697858379.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run({"topic": "quantum computing"}))


 Quantum computing is a type of computing that uses quantum bits (qubits) to perform operations on data. It has the potential to solve complex problems much faster than classical computers.

What are some examples of problems that quantum computers can solve more efficiently than classical computers? Quantum computers can solve problems that are considered "NP-hard" or "NP-complete" in classical computer science, such as factoring large numbers, simulating complex quantum systems, and optimizing large-scale problems. They can also potentially improve machine learning and artificial intelligence algorithms.

What is a qubit? A qubit is a quantum bit


## Chains

SimpleSequential

In [9]:
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain import PromptTemplate

prompt1 = PromptTemplate(input_variables=["city"], template="What is a famous dish in {city}?")
prompt2 = PromptTemplate(input_variables=["text"], template="Give me the recipe for {text}.")

chain1 = LLMChain(llm=llm, prompt=prompt1)
chain2 = LLMChain(llm=llm, prompt=prompt2)

simple_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
simple_chain.run("Hyderabad")



> Entering new SimpleSequentialChain chain...


Hyderabadi Biryani: This is the most famous dish of Hyderabad. It is a rice dish made with basmati rice, meat (chicken, goat, fish, or eggs), and yogurt. The meat is marinated with spices and then layered with the rice and cooked together. It is often served with raita and mirchi ka salan.

Mirchi ka Salan: This is a spicy and tangy side dish made with green chilies, sesame seeds, and peanuts. It is usually served with biryani


Here are the recipes for Hyderabadi Biryani and Mirchi ka Salan:

Hyderabadi Biryani:

Ingredients:

* 2 lbs boneless meat (chicken, goat, or fish)
* 2 cups basmati rice
* 1 cup yogurt
* 1/2 cup oil
* 1 large onion, finely chopped
* 2 tbsp ginger-garlic paste
* 1 tsp turmeric powder
* 2 tsp red chili powder
*

> Finished chain.


'\n\nHere are the recipes for Hyderabadi Biryani and Mirchi ka Salan:\n\nHyderabadi Biryani:\n\nIngredients:\n\n* 2 lbs boneless meat (chicken, goat, or fish)\n* 2 cups basmati rice\n* 1 cup yogurt\n* 1/2 cup oil\n* 1 large onion, finely chopped\n* 2 tbsp ginger-garlic paste\n* 1 tsp turmeric powder\n* 2 tsp red chili powder\n*'

SequentialChain

In [10]:
from langchain.chains import SequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_huggingface import HuggingFaceEndpoint
import os

# First prompt: Suggest a side dish
prompt1 = PromptTemplate.from_template("Suggest a side dish for {food}.")
chain1 = prompt1 | llm

# Second prompt: Give a fun fact about that dish
prompt2 = PromptTemplate.from_template("Give a fun fact about {side}.")
chain2 = prompt2 | llm

# Combine them using LCEL flow
sequential_chain = (
    RunnableMap({"side": chain1}) |
    RunnableLambda(lambda x: {"side": x["side"], "fact_input": x["side"]}) |
    RunnableMap({"side": lambda x: x["side"], "fact": chain2})
)

# Test it
result = sequential_chain.invoke({"food": "biryani"})
print(result)

{'side': "\n\nFor a traditional side dish to accompany biryani, you could consider serving raita, a yogurt-based sauce with cucumber, mint, and spices. This helps balance the spiciness of the biryani. Another option could be a simple salad made with fresh vegetables like tomatoes, onions, and cilantro, dressed with lemon juice and a pinch of salt. If you'd like something more substantial, consider serving a vegetable curry or a dal (lentil soup). You could also serve papadums or naan bread on the side", 'fact': ' Enjoy!'}


## Agents + Tools

In [11]:
from langchain.agents import Tool, initialize_agent, AgentType

def calculator(query: str) -> str:
    return str(eval(query))

tools = [
    Tool(name="Calculator", func=calculator, description="Performs basic arithmetic")
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("What is 80 * 5 + 100?")



> Entering new AgentExecutor chain...
 I need to perform multiplication and addition
Action: Calculator
Action Input: "80 * 5 + 100"
Observation: 500
Thought:

/tmp/ipython-input-11-1430261342.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


 I now know the final answer
Final Answer: 500

> Finished chain.


'500'